In [15]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [16]:
!nvidia-smi

Thu Jun 22 17:38:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   31C    P8    23W / 350W |      8MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
| 39%   

In [8]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [9]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1344.76it/s]


Dataset parquet downloaded and prepared to /home/nlplab11/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 796.34it/s]


In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [11]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/kollama-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so.11.0'), PosixPath('/home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/nlplab11/anaconda3/envs/jyp/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


Loading checkpoint shards: 100%|██████████| 15/15 [00:03<00:00,  4.26it/s]


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [18]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [19]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4194304 || all params: 3668447232 || trainable%: 0.11433458721752714


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [24]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kdysunleo. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.864800
20,3.455500
30,3.298400
40,3.569600
50,3.400300
60,3.488400
70,3.257100
80,3.426500
90,3.359700
100,3.538800


TrainOutput(global_step=500, training_loss=3.394790828704834, metrics={'train_runtime': 419.9929, 'train_samples_per_second': 2.381, 'train_steps_per_second': 1.19, 'total_flos': 4917472162480128.0, 'train_loss': 3.394790828704834, 'epoch': 0.05})

In [36]:
data["train"]["text"][0]

'### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'

In [25]:
print("wow")

wow


In [26]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [27]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/transformers/generation/utils.py:1355: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/transformers/generation/utils.py:1454: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[14831,  6792,    29,  1933, 42731,    34,   202,   202, 14831,  5994,
            29,  1933, 42731,  4210, 25388,   489,    17,  2019,    15,  1933]])

In [28]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [29]:
gen('건강하게 살기 위한 세 가지 방법은?')

### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 건강하게 살기 위한 세 가지 방법은 어떤 종류의 세 가지가 있을 지 예상이 어렵습니다.  

1. 건강하게 살기 위한 세 가지 방법은 일반적으로 체중, 체중, 체중 등으로 구성되는 세 가지 방법입니다.  
2. 건강하게 살기 위한 세 가지 방법은 대체로 체중이나 체중을 관리하기 위한 방법들로 나누어집니다. 
3. 건강하게 살기 위한 세 가지 방법과 체중이나 체중 등에 대한 정보를 교환하기 위한 방안으로는 방법들이 있습니다. 
4. 건강하게 살기 위한 세 가지 방법은 전반적으로 신체 건강 관리에 힘쓰는 방법들로 구별됩니다. 
5. 건강하게 살기 위한 세 가지 방법도 기본적으로 체중, 체중, 신체 건강 관리에 힘쓰는 방법들로 나누어집니다. 
6. 건강을 유지하고, 잘 돌보아 주기 위한 세 가지는 다음과 같습니다. 
7. 건강하게 살기 위한 세 가지 방법은 크게 두 가지 방식으로 나뉩니다. 1. 건강하게 살기 위한 세 가지 방법: 체중과 체중. 체중으로 구성되는 세 가지 방법이 있습니다. 체중을 관리하기 위해서는 체중과 체중이 모두 포함되어 있어야 한다는 점에 유의하여 적당하게 체중을 나누어 체중과 체중을 나누어 건강 관리를 해 나가야 합니다. 체중을 나누


In [30]:
gen('슈카월드가 무엇인가요?')

### 질문: 슈카월드가 무엇인가요?

### 답변: 슈카월은 영국 프로방드에서 3월부터 6월 초까지 활동했던 '호퍼'입니다. 그의 나이 40세입니다. 그의 이름은 '비올라나'(E.A.C.A.R.)입니다. 

슈카월은 프랑스의 피파마, 아탈란타, 툴루즈, 스카리브, 리나(S.A.R.R.R.) 등이 소속돼 있지만, 현재는 '알레데스타'가 되어 있습니다. 

샤방샤방슈카월은 "알로슈(Fruses)", 샤르노(S.A.R.S)), 세리에A(S.A.C.A.R.) 등과 관련된 문헌이 있습니다. 

알로슈는 주로 '오스카' 박하나 베하스(O.Baelus), '오스카(O.C.A.C.A.C.A.R)', '오셔니도비드', '오셔니도비드', '오셔니도비드', '베드너박', '오가사제(D.I.O.C.A.I.O.C.A.I.O


In [31]:
gen('공공성을 측정하기위한 방법은?')

### 질문: 공공성을 측정하기위한 방법은?

### 답변: 한국에는 철골구조가 없어서 건설공사의 중력과 균형을 맞추기 위한 철골조 구조물에 비해, 철골구조물의 깊이가 깊을수록 철골 구조물의 단면을 측정하는 것은 어렵고 정확합니다. 이는 철골조 건축구조물에 비해 더욱 정확합니다. 단면적으로만 봤을 때, 철골조 건축구조물에 비해 정밀도를 향상시킨다는 뜻인 'A2'는 일반적으로 철골구조물의 단면을 고려하지 않고 설계하여 만들어진 공법으로, 건축구조물의 단면상 높이와는 관계 없이 철골 구조물에 비해 정밀한 측정력과 정확한 안전성을 가집니다. 

건물의 각 층마다 두께의 단면상 차이가 있지만, 단면으로 올라가는 철골조 건축구조물은 층고 상한, 바닥 두께 등 철골구조의 단면성에 비해 정밀한 구조체도를 갖게 됩니다.

또한, 단면으로 올라가는 철골조 건축구조는 철골구조 건축물의 단면상에서 가장 높은 단면이 되고, 단면만이 아닌 구조체의 단면을 보여주는 것을 특징으로 하며, 단면은 주로 건물의 높낮이를 측정하여 단면과 단면이 이루는 높이와 단면을 고려하여 산출됩니다.

그리고, 철골구조 건축물에서 단면상보다 단면상


In [32]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

### 답변: 주식시장 거래소는 증권과 증권사, 투신, 증권보험이 가장 널리 사용되고 있지만, 증권사만 표준화된 용어로 사용되고 있습니다. 일반적으로 주식 거래소는 증권 주식과 달리 계좌를 이용하기 때문에 증권 주식에서 발행되는 모든 종목은 증권의 가치와 시장 가치를 함께 평가합니다. 따라서 종목마다 동일한 자산일지를 기준으로 투자하려는 노력을 기울이기 위해, 가장 적합한 상품 선택 방법을 통해 증권 회사와 거래소 모두에게 공통적으로 적합한 상품 선택 방법을 도울 예정입니다. 

중반부에 자산 배분과 시장성이 확인되면, 가장 먼저 증권 업체가 가장 적합한 상품의 선택 방법을 찾습니다. 우선 중세기에 발달한 기술에서 증권 증권 기업이 활용되어왔고, 현재는 증권, 상사 및 예탁결제, ELS 상품 등이 대표적입니다. 이러한 자산 구성을 위해서는 증권 종목 선택 방법과 증권사 선택 방법에 대하여 공부합니다. 증권 회사와 거래소는 각각에 따라 다를 수 있으며, 반드시 둘 중 하나일 것 같습니다. 

투자와 관련된 증권의 종류와 상품을 선택할 수 있는 선택 방식을 고민하고, 증권 회사와 거래소는 서로에게 가장 정확한 정보를 제공할 수 있는 상품들을 찾아보세요. 증권 계좌에 대한 설명을 자세히 통해 증권사에게 가장 적합한 상품 선택 방법을 알려줍니다. 또한, 증권 계좌를 선택하는 방법은 증권사마다 차이가 큰


In [1]:
from datasets import load_dataset
train_dataset = load_dataset('csv', data_files='../preprocess/train.csv')

/home/nlplab11/anaconda3/envs/jyp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlplab11/.cache/huggingface/datasets/csv/default-6ae17a99ca4cc402/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 840.21it/s]
